In [9]:
from __future__ import print_function
import numpy as np
import numba
import awkward

np.random.seed(4)
nrows = 50000

In [2]:
def comb(counts, content):
    out_counts = counts*(counts-1)//2
    out = np.empty((np.sum(out_counts), ) + content.shape[1:])
    i = 0
    n = 0
    for c in counts:
        for i1 in range(i, i+c):
            for i2 in range(i1+1, i+c):
                out[n] = content[i1]+content[i2]
                n += 1
        i += c
    return (out_counts, out)

comb_fast = numba.njit(comb)

def awk_comb(awk):
    c = awk.pairs(same=False)
    return c["0"] + c["1"]

def py_comb(awk):
    counts, content = comb(awk.counts, awk.content)
    return awkward.JaggedArray.fromcounts(counts, content)

def py_comb_fast(awk):
    counts, content = comb_fast(awk.counts, awk.content)
    return awkward.JaggedArray.fromcounts(counts, content)

In [3]:
nwide = 4
counts = np.minimum(np.random.exponential(2, size=nrows).astype(int), 20)
content = np.random.normal(size=np.sum(counts)*nwide).reshape((-1, nwide))
print(content.shape)

awk_vector = awkward.JaggedArray.fromcounts(counts, content)

(77794, 4)


In [7]:
# force pre-compilation of numba functions
_ = py_comb_fast(awk_vector)

%timeit awk_comb(awk_vector)
%timeit py_comb(awk_vector)
%timeit py_comb_fast(awk_vector)
%timeit comb_fast(awk_vector.counts, awk_vector.content)

30.8 ms ± 2.22 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
179 ms ± 3.75 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
15.7 ms ± 515 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
15.4 ms ± 240 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
nwide = 4
counts = np.minimum(np.random.exponential(2, size=nrows).astype(int), 20)
content = np.random.normal(size=np.sum(counts))
print(content.shape)

awk_scalar = awkward.JaggedArray.fromcounts(counts, content)

(77047,)


In [8]:
# force pre-compilation of numba functions
_ = py_comb_fast(awk_scalar)

%timeit awk_comb(awk_scalar)
%timeit py_comb(awk_scalar)
%timeit py_comb_fast(awk_scalar)
%timeit comb_fast(awk_scalar.counts, awk_scalar.content)

18.4 ms ± 613 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
88.4 ms ± 2.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
855 µs ± 20.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
686 µs ± 17.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
